In [1]:
# fmt: off
import urllib
from IPython.display import Markdown as md

### change to reflect your notebook
_nb_loc = "09_deploying/09b_rest.ipynb"
_nb_title = "Predictions using a REST endpoint"

_icons=["https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png", "https://www.tensorflow.org/images/colab_logo_32px.png", "https://www.tensorflow.org/images/GitHub-Mark-32px.png", "https://www.tensorflow.org/images/download_logo_32px.png"]
_links=["https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?" + urllib.parse.urlencode({"name": _nb_title, "download_url": "https://github.com/takumiohym/practical-ml-vision-book-ja/raw/master/"+_nb_loc}), "https://colab.research.google.com/github/takumiohym/practical-ml-vision-book-ja/blob/master/{0}".format(_nb_loc), "https://github.com/takumiohym/practical-ml-vision-book-ja/blob/master/{0}".format(_nb_loc), "https://raw.githubusercontent.com/takumiohym/practical-ml-vision-book-ja/master/{0}".format(_nb_loc)]
md("""<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="{0}"><img src="{4}"/>Run in Vertex AI Workbench</a></td><td><a target="_blank" href="{1}"><img src="{5}" />Run in Google Colab</a></td><td><a target="_blank" href="{2}"><img src="{6}" />View source on GitHub</a></td><td><a href="{3}"><img src="{7}" />Download notebook</a></td></table><br/><br/>""".format(_links[0], _links[1], _links[2], _links[3], _icons[0], _icons[1], _icons[2], _icons[3]))
# fmt: on

<table class="tfo-notebook-buttons" align="left"><td><a target="_blank" href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?name=Predictions+using+a+REST+endpoint&download_url=https%3A%2F%2Fgithub.com%2Ftakumiohym%2Fpractical-ml-vision-book-ja%2Fraw%2Fmaster%2F09_deploying%2F09b_rest.ipynb"><img src="https://raw.githubusercontent.com/GoogleCloudPlatform/practical-ml-vision-book/master/logo-cloud.png"/>Run in Vertex AI Workbench</a></td><td><a target="_blank" href="https://colab.research.google.com/github/takumiohym/practical-ml-vision-book-ja/blob/master/09_deploying/09b_rest.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a></td><td><a target="_blank" href="https://github.com/takumiohym/practical-ml-vision-book-ja/blob/master/09_deploying/09b_rest.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a></td><td><a href="https://raw.githubusercontent.com/takumiohym/practical-ml-vision-book-ja/master/09_deploying/09b_rest.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a></td></table><br/><br/>

# RESTエンドポイントを使用した予測  

このノートブックでは、学習済みでエクスポートされたモデルから始めます。  
このモデルはパブリックの`gs://practical-ml-vision-book-data/flowers_5_trained`に保存されています。

モデルをVeretx AIにデプロイし、様々な方法でモデルを呼び出す方法を示します。

## 環境設定
以下に使用するプロジェクト名を入力してください。

In [ ]:
PROJECT = 'PROJECT_NAME'

Colab環境を利用する場合、以下を実行して必要なモジュールのインストール、ファイルのダウンロード、プロジェクトの設定を行ってください。

In [ ]:
import os

IS_COLAB_BACKEND = 'COLAB_RELEASE_TAG' in os.environ  # this is always set on Colab
if IS_COLAB_BACKEND:
    from google.colab import auth
    auth.authenticate_user()

    # Install dependencies
    !pip install apache-beam==2.38.0

    # Get files
    BASE_PATH = "https://raw.githubusercontent.com/takumiohym/practical-ml-vision-book-ja/main/09_deploying"
    !wget {BASE_PATH}/vertex_deploy.sh
    !sudo chmod 755 ./vertex_deploy.sh

    !gcloud config set project {PROJECT}

## Google Cloud Storageバケットの作成
以下のBUCKETの値を使用可能なGSCバケット名に変更して進めてください。<br>
使用できるバケットがない場合は、以下の二行目のコマンドをコメントアウトし、gsutil mbコマンドを実行して作成してください。

In [ ]:
BUCKET='BUCKET_NAME' # Specify your GCS bucket name
# !gsutil mb -l us-central1 -p {PROJECT} gs://{BUCKET} 

## RESTエンドポイントの作成

`vertex_deploy.sh` コマンドで、Vertex AIへ学習済みのモデルのアップロード（`gcloud ai models upload`）とデプロイ（`gcloud ai endpoints deploy-model`）を行います。

実行には10分ほどの時間がかかります。

In [2]:
!cat ./vertex_deploy.sh

#!/bin/bash

REGION="us-central1"  # make sure you have GPU/TPU quota in this region
ENDPOINT_NAME="flowers_endpoint"
MODEL_NAME="flowers"
MODEL_LOCATION="gs://practical-ml-vision-book-data/flowers_5_trained"
IMAGE_URI="us-docker.pkg.dev/vertex-ai/prediction/tf2-cpu.2-4:latest"

for i in "$@"
do
case $i in
        -r=*|--region=*) REGION="${i#*=}"; shift ;;
        -e=*|--endpoint_name=*) ENDPOINT_NAME="${i#*=}"; shift ;;
        -m=*|--model_name=*) MODEL_NAME="${i#*=}"; shift ;;
        -l=*|--model_location=*) MODEL_LOCATION="${i#*=}"; shift ;;
        -i=*|--image_uri=*) IMAGE_URI="${i#*=}"; shift ;;
        *) echo "Unknown parameter passed: $1"; exit 1 ;;
esac
done

echo "Deploying model $MODEL_NAME"

if [[ $(gcloud ai endpoints list --region=$REGION --format="value(display_name)" | grep $ENDPOINT_NAME) ]]; then
    echo "The endpoint named $ENDPOINT_NAME already exists."
else
    # Create endpoint.
    echo "Creating $ENDPOINT_NAME endpoint now."
    gcloud ai endpoints create \

In [3]:
!./vertex_deploy.sh

Deploying model flowers
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Creating flowers_endpoint endpoint now.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [5503301506587164672]...done.                            
Created Vertex AI endpoint: projects/849204435784/locations/us-central1/endpoints/7290157916340879360.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The endpoint_id is 7290157916340879360
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Uploading flowers model now.
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [3201962097000841216]...done.                            
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
The model_id is 3925815063067230208
Deploying model now
Using endpoint [https://us-central1-aiplatform.googleapis.com/]
Waiting for operation [7809144515800858624]...done.                            
Deployed a model to t

## 以下のセルを変更する

上記のコマンドから出力された`endpoint_id`とデプロイされた`model_id`を下記にコピーしてください。

In [4]:
# CHANGE THESE TO REFLECT WHERE YOU DEPLOYED THE MODEL
import os
os.environ['ENDPOINT_ID'] = '7290157916340879360' # CHANGE
os.environ['MODEL_ID'] = '3925815063067230208' # CHANGE
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = 'us-central1'

## Online Prediction: gcloud コマンド
Vertex AI Predictionを利用して、デプロイしたモデルにリクエストを送り、推論結果を取得します。

モデルの入力に合わせて以下のjsonファイルを作成し、`gcloud ai endpoints predict`コマンドでリクエストを送ります。

In [5]:
%%writefile request.json
{
    "instances": [
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}

Overwriting request.json


In [6]:
%%bash
gcloud ai endpoints predict ${ENDPOINT_ID} \
--region=${REGION} \
--json-request=request.json \
--format=json

{
  "deployedModelId": "9036406681621233664",
  "model": "projects/849204435784/locations/us-central1/models/3925815063067230208",
  "modelDisplayName": "flowers",
  "predictions": [
    {
      "flower_type_int": 1,
      "flower_type_str": "dandelion",
      "probability": 0.619152546
    },
    {
      "flower_type_int": 1,
      "flower_type_str": "dandelion",
      "probability": 0.999984384
    },
    {
      "flower_type_int": 0,
      "flower_type_str": "daisy",
      "probability": 0.995082855
    },
    {
      "flower_type_int": 4,
      "flower_type_str": "tulips",
      "probability": 0.975185812
    },
    {
      "flower_type_int": 4,
      "flower_type_str": "tulips",
      "probability": 0.954917
    }
  ]
}


Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]


## Online Prediction: HTTPリクエスト

同様のことは、直接HTTPリクエストを定義して送信することでも行うことができます。（また、各プログラミング言語のGoogle Cloud SDKを使用することも可能です）

In [7]:
# Invoke from Python.
import json
from oauth2client.client import GoogleCredentials
import requests

PROJECT = os.environ['PROJECT']
REGION = os.environ['REGION']
ENDPOINT_ID = os.environ['ENDPOINT_ID']

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/endpoints/{}:predict".format(
    REGION, PROJECT, REGION, ENDPOINT_ID)
headers = {"Authorization": "Bearer " + token }
data = {
    "instances": [
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"
        },
        {
            "filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
        }
    ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{\n  "predictions": [\n    {\n      "flower_type_int": 1,\n      "probability": 0.619152546,\n      "flower_type_str": "dandelion"\n    },\n    {\n      "flower_type_int": 1,\n      "flower_type_str": "dandelion",\n      "probability": 0.999984384\n    },\n    {\n      "flower_type_str": "daisy",\n      "probability": 0.995082855,\n      "flower_type_int": 0\n    },\n    {\n      "flower_type_int": 4,\n      "flower_type_str": "tulips",\n      "probability": 0.975185812\n    },\n    {\n      "flower_type_str": "tulips",\n      "probability": 0.954917,\n      "flower_type_int": 4\n    }\n  ],\n  "deployedModelId": "9036406681621233664",\n  "model": "projects/849204435784/locations/us-central1/models/3925815063067230208",\n  "modelDisplayName": "flowers"\n}\n'


## Batch Prediction
続いて、Vertex AIを利用してバッチでの推論を行います。

以下のようにJson Linesで入力データを定義し、送信します。


In [8]:
%%writefile batchinputs.jsonl
{"filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg"}
{"filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg"}
{"filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg"}
{"filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg"}
{"filenames": "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"}

Overwriting batchinputs.jsonl


In [9]:
%%bash
gsutil cp batchinputs.jsonl gs://${BUCKET}

Copying file://batchinputs.jsonl [Content-Type=application/octet-stream]...
/ [1 files][  555.0 B/  555.0 B]                                                
Operation completed over 1 objects/555.0 B.                                      


In [10]:
# Invoke from Python.
import json
from oauth2client.client import GoogleCredentials
import requests

PROJECT = os.environ['PROJECT']
REGION = os.environ['REGION']
ENDPOINT_ID = os.environ['ENDPOINT_ID']
MODEL_ID = os.environ['MODEL_ID']
BUCKET = os.environ['BUCKET'] # used for staging

BATCH_JOB_NAME = "batch_pred_job"
INPUT_FORMAT = "jsonl"
INPUT_URI = "gs://{}/batchinputs.jsonl".format(BUCKET)
OUTPUT_DIRECTORY = "gs://{}/batch_predictions".format(BUCKET)
MACHINE_TYPE = "n1-standard-2"
STARTING_REPLICA_COUNT = 1
BATCH_SIZE = 64

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/batchPredictionJobs".format(
    REGION, PROJECT, REGION
)
headers = {"Authorization": "Bearer " + token}
data = {
    "displayName": BATCH_JOB_NAME,
    "model": "projects/{}/locations/{}/models/{}".format(
        PROJECT, REGION, MODEL_ID
    ),
    "inputConfig": {
        "instancesFormat": INPUT_FORMAT,
        "gcsSource": {
            "uris": [INPUT_URI],
        },
    },
    "outputConfig": {
        "predictionsFormat": "jsonl",
        "gcsDestination": {
            "outputUriPrefix": OUTPUT_DIRECTORY,
        },
    },
    "dedicatedResources" : {
        "machineSpec" : {
            "machineType": MACHINE_TYPE
        },
        "startingReplicaCount": STARTING_REPLICA_COUNT
    },
    "manualBatchTuningParameters": {
        "batch_size": BATCH_SIZE,
    }
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{\n  "name": "projects/849204435784/locations/us-central1/batchPredictionJobs/8101512010610507776",\n  "displayName": "batch_pred_job",\n  "model": "projects/849204435784/locations/us-central1/models/3925815063067230208",\n  "inputConfig": {\n    "instancesFormat": "jsonl",\n    "gcsSource": {\n      "uris": [\n        "gs://takumi-misc/batchinputs.jsonl"\n      ]\n    }\n  },\n  "outputConfig": {\n    "predictionsFormat": "jsonl",\n    "gcsDestination": {\n      "outputUriPrefix": "gs://takumi-misc/batch_predictions"\n    }\n  },\n  "dedicatedResources": {\n    "machineSpec": {\n      "machineType": "n1-standard-2"\n    },\n    "startingReplicaCount": 1\n  },\n  "manualBatchTuningParameters": {\n    "batchSize": 64\n  },\n  "state": "JOB_STATE_PENDING",\n  "createTime": "2022-07-17T21:27:20.654512Z",\n  "updateTime": "2022-07-17T21:27:20.654512Z",\n  "modelVersionId": "1"\n}\n'


## Apache BeamからのOnline Predictionの呼び出し

最後に、Vertex AIのOnline PredictionをApache Beamのパイプラインから呼び出す方法を確認します。

In [11]:
import apache_beam as beam
import json
from oauth2client.client import GoogleCredentials
import requests

class ModelPredict:
    def __init__(self, project, region, endpoint_id):
        self._api = "https://{}-aiplatform.googleapis.com/v1/projects/{}/locations/{}/endpoints/{}:predict".format(
            region, project, region, endpoint_id)   
        
    def __call__(self, filenames):        
        token = GoogleCredentials.get_application_default().get_access_token().access_token
        if isinstance(filenames, str):
            # Only one element, put it into a batch of 1.
            data = {
                "instances": [
                    {"filenames": filenames}
                ]
            }
        else:
            data = {
                "instances": []
            }
            for f in filenames:
                data["instances"].append({
                    "filenames" : f
                })
        # print(data)
        headers = {"Authorization": "Bearer " + token }
        response = requests.post(self._api, json=data, headers=headers)
        response = json.loads(response.content.decode("utf-8"))
        # print(response)
        if isinstance(filenames, str):
            result = response["predictions"][0]
            result["filename"] = filenames
            yield result
        else:
            for (a,b) in zip(filenames, response["predictions"]):
                result = b
                result["filename"] = a
                yield result


PROJECT = os.environ['PROJECT']
REGION = os.environ['REGION']
ENDPOINT_ID = os.environ['ENDPOINT_ID']

with beam.Pipeline() as p:    
    (p 
     | "input" >> beam.Create([
        "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg",
        "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg",
        "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg",
        "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg",
        "gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg"
     ]) 
     | "batch" >> beam.BatchElements(min_batch_size=2, max_batch_size=3)
     | "addpred" >> beam.FlatMap(ModelPredict(PROJECT, REGION, ENDPOINT_ID))
     | "write" >> beam.Map(print)
    )

{'flower_type_str': 'dandelion', 'probability': 0.619152427, 'flower_type_int': 1, 'filename': 'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9818247_e2eac18894.jpg'}
{'flower_type_int': 1, 'probability': 0.999984384, 'flower_type_str': 'dandelion', 'filename': 'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/dandelion/9853885425_4a82356f1d_m.jpg'}
{'flower_type_str': 'daisy', 'flower_type_int': 0, 'probability': 0.995082855, 'filename': 'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/daisy/9299302012_958c70564c_n.jpg'}
{'flower_type_int': 4, 'flower_type_str': 'tulips', 'probability': 0.975185931, 'filename': 'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8733586143_3139db6e9e_n.jpg'}
{'probability': 0.954917, 'flower_type_str': 'tulips', 'flower_type_int': 4, 'filename': 'gs://practical-ml-vision-book-data/flowers_5_jpeg/flower_photos/tulips/8713397358_0505cc0176_n.jpg'}


## License
Copyright 2022 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.